# Scratchwork

In [1]:
from finite_algebras import *
from cayley_table import *
from permutations import *

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [3]:
ex = Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  15 example algebras are available.
  Use "get_example(INDEX)" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-group

## Subalgebras of a Semigroup

In [6]:
sg = ex.get_example(11)
sg.about()


Semigroup: Example 1.4.1
Instance ID: 140257409298640
Description: See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
Order: 6
Elements: ['a', 'b', 'c', 'd', 'e', 'f']
Identity: None
Associative? Yes
Commutative? No
Has Inverses? No
Cayley Table (showing indices):
[[0, 3, 0, 3, 0, 3],
 [1, 4, 1, 4, 1, 4],
 [2, 5, 2, 5, 2, 5],
 [3, 0, 3, 0, 3, 0],
 [4, 1, 4, 1, 4, 1],
 [5, 2, 5, 2, 5, 2]]


In [7]:
alg = sg
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(sg, partitions)


Subalgebras of <Semigroup:Example 1.4.1, ID:140257409298640>
  There are 4 unique subalgebras, up to isomorphisms, out of 10 total subalgebras
  as shown by the partitions below:

3 Commutative Groups of order 2:
      Example 1.4.1_subalgebra_0: ['c', 'f'] with identity 'c'
      Example 1.4.1_subalgebra_2: ['a', 'd'] with identity 'a'
      Example 1.4.1_subalgebra_3: ['b', 'e'] with identity 'e'

1 Semigroup of order 3:
      Example 1.4.1_subalgebra_1: ['a', 'c', 'e']

3 Semigroups of order 2:
      Example 1.4.1_subalgebra_4: ['c', 'e']
      Example 1.4.1_subalgebra_8: ['a', 'e']
      Example 1.4.1_subalgebra_9: ['a', 'c']

3 Semigroups of order 4:
      Example 1.4.1_subalgebra_5: ['a', 'c', 'd', 'f']
      Example 1.4.1_subalgebra_6: ['b', 'c', 'e', 'f']
      Example 1.4.1_subalgebra_7: ['a', 'b', 'd', 'e']



## Subgroups of an Alternating Group

In [8]:
a4 = ex.get_example(0)
a4.about()


Group: A4
Instance ID: 140257132515600
Description: Alternating group on 4 letters (AKA Tetrahedral group)
Order: 12
Identity: ()
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0      ()      ()       1
      1 (1 3 2) (1 2 3)       3
      2 (1 2 3) (1 3 2)       3
      3 (0 1)(2 3) (0 1)(2 3)       2
      4 (0 1 2) (0 2 1)       3
      5 (0 1 3) (0 3 1)       3
      6 (0 3 1) (0 1 3)       3
      7 (0 3 2) (0 2 3)       3
      8 (0 3)(1 2) (0 3)(1 2)       2
      9 (0 2 3) (0 3 2)       3
     10 (0 2 1) (0 1 2)       3
     11 (0 2)(1 3) (0 2)(1 3)       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4],
 [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7],
 [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8],
 [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1],
 [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10],
 [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9],
 [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2],
 [8, 6, 7, 11, 9, 10, 1, 2, 0, 

In [10]:
alg = a4
alg_subs = alg.proper_subalgebras(divisors_only=False, include_inverses=False)
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Group:A4, ID:140257132515600>
  There are 3 unique subalgebras, up to isomorphisms, out of 8 total subalgebras
  as shown by the partitions below:

4 Commutative Groups of order 3 with identity '()':
      A4_subalgebra_0: ['()', '(1 3 2)', '(1 2 3)']
      A4_subalgebra_1: ['()', '(0 1 3)', '(0 3 1)']
      A4_subalgebra_2: ['()', '(0 3 2)', '(0 2 3)']
      A4_subalgebra_3: ['()', '(0 1 2)', '(0 2 1)']

3 Commutative Groups of order 2 with identity '()':
      A4_subalgebra_4: ['()', '(0 1)(2 3)']
      A4_subalgebra_5: ['()', '(0 3)(1 2)']
      A4_subalgebra_7: ['()', '(0 2)(1 3)']

1 Commutative Normal Group of order 4 with identity '()':
      A4_subalgebra_6: ['()', '(0 1)(2 3)', '(0 3)(1 2)', '(0 2)(1 3)']



In [11]:
f4 = ex.get_example(9)
f4.about()


Field: F4
Instance ID: 140257409289168
Description: Field with 4 elements (from Wikipedia)
Order: 4
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


In [12]:
alg = f4
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Field:F4, ID:140257409289168>
  There are 1 unique subalgebras, up to isomorphisms, out of 3 total subalgebras
  as shown by the partitions below:

3 Commutative Normal Groups of order 2 with identity '0':
      F4_subalgebra_0: ['0', '1+a']
      F4_subalgebra_1: ['0', 'a']
      F4_subalgebra_2: ['0', '1']



In [13]:
ex6 = ex.get_example(12)
ex6.about()


Ring: Ex6
Instance ID: 140257132572624
Description: Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html
Order: 4
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       a       a       2
      2       b       b       2
      3       c       c       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: None
Mult. Commutative? No
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]]


In [14]:
alg = ex6
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Ring:Ex6, ID:140257132572624>
  There are 1 unique subalgebras, up to isomorphisms, out of 3 total subalgebras
  as shown by the partitions below:

3 Commutative Normal Groups of order 2 with identity '0':
      Ex6_subalgebra_0: ['0', 'b']
      Ex6_subalgebra_1: ['0', 'c']
      Ex6_subalgebra_2: ['0', 'a']



In [19]:
psr = generate_powerset_group(4)
psr.about(max_size=16)


Group: PS4
Instance ID: 140257409395728
Description: Autogenerated Group on the powerset of 4 elements, with symmetric difference operator
Order: 16
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4     {3}     {3}       2
      5  {0, 1}  {0, 1}       2
      6  {0, 2}  {0, 2}       2
      7  {0, 3}  {0, 3}       2
      8  {1, 2}  {1, 2}       2
      9  {1, 3}  {1, 3}       2
     10  {2, 3}  {2, 3}       2
     11 {0, 1, 2} {0, 1, 2}       2
     12 {0, 1, 3} {0, 1, 3}       2
     13 {0, 2, 3} {0, 2, 3}       2
     14 {1, 2, 3} {1, 2, 3}       2
     15 {0, 1, 2, 3} {0, 1, 2, 3}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 0, 5, 6, 7, 2, 3, 4, 11, 12, 13, 8, 9, 10, 15, 14],
 [2, 5, 0, 8, 9, 1, 11, 12, 3, 4, 14, 6, 7, 15, 10, 13],
 [3, 6, 8, 0, 10, 1

In [26]:
alg = psr
%time alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)

CPU times: user 7.88 s, sys: 23.8 ms, total: 7.9 s
Wall time: 7.92 s

Subalgebras of <Group:PS4, ID:140257409395728>
  There are 3 unique subalgebras, up to isomorphisms, out of 65 total subalgebras
  as shown by the partitions below:

35 Commutative Normal Groups of order 4 with identity '{}':
      PS4_subalgebra_0: ['{}', '{0}', '{1, 2}', '{0, 1, 2}']
      PS4_subalgebra_2: ['{}', '{2}', '{1, 3}', '{1, 2, 3}']
      PS4_subalgebra_7: ['{}', '{0}', '{1, 2, 3}', '{0, 1, 2, 3}']
      PS4_subalgebra_9: ['{}', '{1}', '{2}', '{1, 2}']
      PS4_subalgebra_11: ['{}', '{0, 2}', '{0, 1, 3}', '{1, 2, 3}']
      PS4_subalgebra_12: ['{}', '{2}', '{0, 3}', '{0, 2, 3}']
      PS4_subalgebra_13: ['{}', '{3}', '{0, 1}', '{0, 1, 3}']
      PS4_subalgebra_14: ['{}', '{0}', '{3}', '{0, 3}']
      PS4_subalgebra_15: ['{}', '{0}', '{1, 3}', '{0, 1, 3}']
      PS4_subalgebra_16: ['{}', '{0, 1}', '{0, 3}', '{1, 3}']
      PS4_subalgebra_17: ['{}', '{0, 2}', '{1, 3}', '{0, 1, 2, 3}']
      PS4_subalgebra

In [27]:
alg = psr
%time alg_subs = alg.proper_subalgebras(divisors_only=False, include_inverses=False)
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)

CPU times: user 32.4 s, sys: 87.5 ms, total: 32.5 s
Wall time: 32.6 s

Subalgebras of <Group:PS4, ID:140257409395728>
  There are 3 unique subalgebras, up to isomorphisms, out of 65 total subalgebras
  as shown by the partitions below:

35 Commutative Normal Groups of order 4 with identity '{}':
      PS4_subalgebra_0: ['{}', '{0}', '{1, 2}', '{0, 1, 2}']
      PS4_subalgebra_2: ['{}', '{2}', '{1, 3}', '{1, 2, 3}']
      PS4_subalgebra_7: ['{}', '{0}', '{1, 2, 3}', '{0, 1, 2, 3}']
      PS4_subalgebra_9: ['{}', '{1}', '{2}', '{1, 2}']
      PS4_subalgebra_11: ['{}', '{0, 2}', '{0, 1, 3}', '{1, 2, 3}']
      PS4_subalgebra_12: ['{}', '{2}', '{0, 3}', '{0, 2, 3}']
      PS4_subalgebra_13: ['{}', '{3}', '{0, 1}', '{0, 1, 3}']
      PS4_subalgebra_14: ['{}', '{0}', '{3}', '{0, 3}']
      PS4_subalgebra_15: ['{}', '{0}', '{1, 3}', '{0, 1, 3}']
      PS4_subalgebra_16: ['{}', '{0, 1}', '{0, 3}', '{1, 3}']
      PS4_subalgebra_17: ['{}', '{0, 2}', '{1, 3}', '{0, 1, 2, 3}']
      PS4_subalgebr

In [28]:
# %time foo1 = psr.closed_subsets_of_elements(False)

In [29]:
# {len(x) for x in foo1}

In [30]:
# %time foo2 = psr.closed_subsets_of_elements(True)

In [31]:
# {len(x) for x in foo2}

In [32]:
# %time psr_subs = psr.unique_proper_subgroups()

In [33]:
# [psr_sub.order for psr_sub in psr_subs]

In [34]:
q8 = ex.get_example(13)
q8.about()


Group: Q8
Instance ID: 140257409299664
Description: Quaternion Group
Order: 8
Identity: 1
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0       1       1       1
      1       i      -i       4
      2      -1      -1       2
      3      -i       i       4
      4       j      -j       4
      5       k      -k       4
      6      -j       j       4
      7      -k       k       4
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 0, 7, 4, 5, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 0, 1, 2, 5, 6, 7, 4],
 [4, 5, 6, 7, 2, 3, 0, 1],
 [5, 6, 7, 4, 1, 2, 3, 0],
 [6, 7, 4, 5, 0, 1, 2, 3],
 [7, 4, 5, 6, 3, 0, 1, 2]]


In [35]:
alg = q8
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Group:Q8, ID:140257409299664>
  There are 2 unique subalgebras, up to isomorphisms, out of 4 total subalgebras
  as shown by the partitions below:

3 Commutative Normal Groups of order 4 with identity '1':
      Q8_subalgebra_0: ['1', 'i', '-1', '-i']
      Q8_subalgebra_1: ['1', '-1', 'k', '-k']
      Q8_subalgebra_2: ['1', '-1', 'j', '-j']

1 Commutative Normal Group of order 2 with identity '1':
      Q8_subalgebra_3: ['1', '-1']



In [36]:
sd16 = ex.get_example(14)
sd16.about(max_size=16)


Group: SD16
Instance ID: 140257409300176
Description: Semidihedral group of order 16
Order: 16
Identity: e
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       t       t       2
      2       s  ts^2ts       8
      3      st   s^2ts       4
      4     s^2   ts^2t       4
      5    s^2t    s^2t       2
      6     tst  tsts^2       8
      7      ts   sts^2       4
      8    tsts    tsts       2
      9     sts     sts       2
     10  tsts^2     tst       8
     11   s^2ts      st       4
     12   ts^2t     s^2       4
     13    ts^2    ts^2       2
     14  ts^2ts       s       8
     15   sts^2      ts       4
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 0, 7, 6, 13, 12, 3, 2, 9, 8, 15, 14, 5, 4, 11, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 1],
 [3, 2, 9, 8, 15, 14, 5, 4, 11, 10, 1, 0, 7, 6, 13, 12],
 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 

In [37]:
alg = sd16
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Group:SD16, ID:140257409300176>
  There are 6 unique subalgebras, up to isomorphisms, out of 13 total subalgebras
  as shown by the partitions below:

3 Commutative Groups of order 4 with identity 'e':
      SD16_subalgebra_0: ['e', 'ts', 'tsts', 'sts^2']
      SD16_subalgebra_10: ['e', 'st', 'tsts', 's^2ts']
      SD16_subalgebra_11: ['e', 's^2', 'tsts', 'ts^2t']

5 Commutative Groups of order 2 with identity 'e':
      SD16_subalgebra_1: ['e', 'sts']
      SD16_subalgebra_2: ['e', 't']
      SD16_subalgebra_4: ['e', 'ts^2']
      SD16_subalgebra_7: ['e', 'tsts']
      SD16_subalgebra_12: ['e', 's^2t']

1 Commutative Normal Group of order 8 with identity 'e':
      SD16_subalgebra_3: ['e', 's', 's^2', 'tst', 'tsts', 'tsts^2', 'ts^2t', 'ts^2ts']

2 Commutative Groups of order 4 with identity 'e':
      SD16_subalgebra_5: ['e', 't', 'tsts', 'sts']
      SD16_subalgebra_9: ['e', 's^2t', 'tsts', 'ts^2']

1 Normal Group of order 8 with identity 'e':
      SD16_subalgebra_6

In [ ]:
def ring_closure(ring, subset_of_elements):

    result = set(subset_of_elements)

    # Include additive inverses
    for elem in subset_of_elements:
        result.add(ring.inv(elem))

    # Add the products of all possible pairs
    for pair in it.product(result, result):
        result.add(ring.op(*pair))

    # If the input set of elements increased, recurse ...
    if len(result) > len(subset_of_elements):
        return ring.closure(list(result), include_inverses)

    # ...otherwise, stop and return the result
    else:
        return list(result)

In [ ]:
ex6.about()

In [ ]:
ring_closure(sg, ['d', 'c'], True)